In [ ]:
import pandas as pd
import numpy as np
import warnings

warnings.simplefilter("ignore")


- **Definisikan Fungsi**

In [7]:
# Fungsi untuk menampilkan seluruh data teks pada kolom
def showAll(column):
    for value in column:
        print(value)


# Fungsi untuk menampilkan data pada kategori tertentu saja
def showOneCategory(typeColumn, textColumn, columnType):
    for text, category in zip(textColumn, typeColumn):
        if category == columnType:
            print(text)


# Fungsi untuk memeriksa apakah suatu kata ada dalam array atau tidak (untuk menghindari duplikasi)
def check(array, word):
    return word not in array


# Fungsi untuk menggabungkan semua data hasil pra-pemrosesan menjadi daftar kata tunggal (tanpa duplikasi)
def termMerging(textColumn, termList):
    for text in textColumn:
        for term in text:
            if check(termList, term):
                termList.append(term)


# Fungsi untuk menghitung frekuensi kemunculan setiap kata dalam setiap dokumen
def countTF(textColumn, typeColumn, termList, TFList):
    for text, category in zip(textColumn, typeColumn):
        docList = [text.count(term) for term in termList]
        docList.append(category)
        TFList.append(docList)


# Fungsi untuk mengubah nilai yang lebih besar dari 1 menjadi 1 (untuk perhitungan DF)
def changeValue(TFList, prepareDFList):
    for doc in TFList:
        dataList = [min(term, 1) for term in doc[:-1]]
        prepareDFList.append(dataList)


# Fungsi untuk menghitung dalam berapa banyak dokumen suatu kata muncul (DF)
def countDF(TFList, DFList):
    for term in zip(*TFList):
        DFList.append(sum(term))


## **TF-RF Dataset 1 (Tanpa Augmentasi)**

**1. Stemmed Dataset**

In [ ]:
load_path = "../dataset/INA_TweetsPPKM_Preprocessed_DS1.csv"
# Menentukan path file dataset yang akan di-load

df = pd.read_csv(load_path, sep="\t")
# Membaca dataset dari file CSV menggunakan pandas

df["tweet"] = df["tweet"].apply(lambda word: eval(word))
# Menggunakan fungsi `apply` pada kolom "tweet" untuk menerapkan fungsi lambda pada setiap kata
# Fungsi lambda digunakan untuk mengkonversi string kata-kata menjadi list kata-kata dengan menggunakan fungsi `eval`

df.sample(5)
# Menampilkan 10 baris pertama dari dataframe yang sudah diubah


In [ ]:
df.drop(['date','user'],axis=1,inplace=True)

In [ ]:
term = []
# Membuat list kosong untuk menyimpan semua term

termMerging(termList=term, textColumn=df["tweet"])
# Memanggil fungsi `termMerging` dengan parameter `termList` yang merupakan list kosong yang akan diisi dengan semua term
# Parameter `textColumn` adalah kolom "tweet" dari dataframe `df`
# Fungsi `termMerging` akan menggabungkan semua term dari kolom "tweet" ke dalam list `term` tanpa adanya term yang berulang


In [ ]:
term_frequency = []
# Membuat list kosong untuk menyimpan frekuensi term dalam setiap dokumen

countTF(df["tweet"], df["sentiment"], term, term_frequency)
# Memanggil fungsi `countTF` dengan parameter sebagai berikut:
# - `textColumn`: kolom "tweet" dari dataframe `df` yang berisi teks dalam setiap dokumen
# - `typeColumn`: kolom "sentiment" dari dataframe `df` yang berisi label sentimen untuk setiap dokumen
# - `termList`: list yang berisi semua term yang telah dihasilkan sebelumnya
# - `TFList`: list yang akan diisi dengan frekuensi term dalam setiap dokumen dan label sentimennya

print(len(term_frequency))
# Menampilkan panjang dari list `term_frequency`, yang mengindikasikan jumlah dokumen yang diproses


In [ ]:
tf_forDF = []
# Membuat list kosong untuk menyimpan frekuensi term dalam setiap dokumen dengan nilai yang telah diubah

changeValue(term_frequency, tf_forDF)
# Memanggil fungsi `changeValue` dengan parameter sebagai berikut:
# - `TFList`: list `term_frequency` yang berisi frekuensi term dalam setiap dokumen dan label sentimennya
# - `prepareDFList`: list yang akan diisi dengan frekuensi term dalam setiap dokumen dengan nilai yang telah diubah

print(len(tf_forDF))
# Menampilkan panjang dari list `tf_forDF`, yang mengindikasikan jumlah dokumen yang diproses setelah nilai yang lebih besar dari 1 diubah menjadi 1


In [ ]:
document_frequency = []
# Membuat list kosong untuk menyimpan frekuensi kemunculan setiap term dalam dokumen

countDF(tf_forDF, document_frequency)
# Memanggil fungsi `countDF` dengan parameter sebagai berikut:
# - `TFList`: list `tf_forDF` yang berisi frekuensi term dalam setiap dokumen dengan nilai yang telah diubah
# - `DFList`: list yang akan diisi dengan frekuensi kemunculan setiap term dalam dokumen

print(len(document_frequency))
# Menampilkan panjang dari list `document_frequency`, yang mengindikasikan jumlah term yang muncul dalam dokumen


### **_Find value for RF & TF-RF_**

<ol>
  <li>RF</li>
  <li>TF-RF</li>

</ol>


In [ ]:
# method to count TF in each document category
def count_tf_category(dataTableCategory, termList, textColumn):
    tf_spam_list = []
    for i in range(len(dataTableCategory)):
        doclist = []
        for j in termList:
            count = 0
            for k in range(len(textColumn[i])):
                if j == textColumn[i][k]:
                    count += 1
            doclist.append(count)
        doclist.append(dataTableCategory["sentiment"][i])
        tf_spam_list.append(doclist)
    return tf_spam_list


In [ ]:
# splitting tweets that only contain positive sentiment
df_positive = df[df["sentiment"] != 0].reset_index(drop=True)
df_positive.head()


In [ ]:
# splitting tweets that only contain negative sentiment
df_negative = df[df["sentiment"] != 1].reset_index(drop=True)
df_negative.head()


In [ ]:
# count tf value in tweets with positive sentiment
tf_positive_rf = count_tf_category(df_positive, term, df_positive["tweet"])
print(len(tf_positive_rf))


In [ ]:
# count tf value in tweets with negative sentiment
tf_negative_rf = count_tf_category(df_negative, term, df_negative["tweet"])
print(len(tf_negative_rf))


In [ ]:
tf_positive_df = []
changeValue(tf_positive_rf, tf_positive_df)
print(len(tf_positive_df))


In [ ]:
tf_negative_df = []
changeValue(tf_negative_rf, tf_negative_df)
print(len(tf_negative_df))


In [ ]:
nij = []
countDF(tf_positive_df, nij)
print(len(nij))


In [ ]:
nij_aksen = []
countDF(tf_negative_df, nij_aksen)
print(len(nij_aksen))


In [ ]:
# determining the maximum value between 1 and the value of nij accent
max_numerator = []
for i in range(len(nij_aksen)):
    temp = 0
    temp = max(1, nij_aksen[i])
    max_numerator.append(temp)
print(max_numerator)


In [ ]:
import math

# find rf value
rf_list = []
for i in range(len(nij)):
    temp = 0
    temp = math.log10(2 + (nij[i] / max_numerator[i]))
    temp = round(temp, 5)
    rf_list.append(temp)
print(rf_list)


In [ ]:
# Combining the values of TF for spam and non-spam.
tf_forRF = []

for i in range(len(tf_positive_rf)):
    tf_forRF.append(tf_positive_rf[i])

for i in range(len(tf_negative_rf)):
    tf_forRF.append(tf_negative_rf[i])

print(len(tf_forRF))


In [ ]:
tf_rf = []
for i in range(len(tf_forRF)):
    doclist = []
    for j in range(len(tf_forRF[0])):
        if j == len(tf_forRF[0]) - 1:
            temp = tf_forRF[i][j]
            doclist.append(temp)
        else:
            temp = rf_list[j] * tf_forRF[i][j]
            doclist.append(temp)
    tf_rf.append(doclist)
print(len(tf_rf))


- **Save To CSV**


In [ ]:
columnName = term
columnName.append('sentiment')

In [ ]:
data = pd.DataFrame(tf_rf, columns=columnName)
data.to_csv("../dataset/INA_TweetsPPKM_TFRF_DS1.csv", index=False, sep="\t")


**2. Non Stemmed Dataset**

In [5]:
load_path = "../dataset/INA_TweetsPPKM_Preprocessed_DS1_NoStem.csv"
# Menentukan path file dataset yang akan di-load

df = pd.read_csv(load_path, sep="\t")
# Membaca dataset dari file CSV menggunakan pandas

df["tweet"] = df["tweet"].apply(lambda word: eval(word))
# Menggunakan fungsi `apply` pada kolom "tweet" untuk menerapkan fungsi lambda pada setiap kata
# Fungsi lambda digunakan untuk mengkonversi string kata-kata menjadi list kata-kata dengan menggunakan fungsi `eval`

df.sample(5)
# Menampilkan 10 baris pertama dari dataframe yang sudah diubah

,date,user,tweet,sentiment
4612,2021-07-10 05:02:18+00:00,_nadiyamd,"[jangan, menjauh, lalu, pergi, seenaknya, hati...",0
1421,2021-08-10 02:33:04+00:00,Luthfimuqodas1,"[pagi, indah, kulub, cau, meski, terasa, sakit...",1
5220,2021-07-05 10:33:56+00:00,Rtuuu01,"[pak, sudahi, bisnis, mencekik, rakyat]",0
715,2021-08-25 08:22:57+00:00,aad_taufiq,"[dodol, lekas, seminggu, kok, yo, parani, pakp...",0
2803,2021-07-25 17:00:42+00:00,FinzaHabib,"[pemerintah, menurunkan, aturan, rakyat, kecil...",0


In [6]:
df.drop(['date','user'],axis=1,inplace=True,errors='ignore')

In [8]:
term = []
# Membuat list kosong untuk menyimpan semua term

termMerging(termList=term, textColumn=df["tweet"])
# Memanggil fungsi `termMerging` dengan parameter `termList` yang merupakan list kosong yang akan diisi dengan semua term
# Parameter `textColumn` adalah kolom "tweet" dari dataframe `df`
# Fungsi `termMerging` akan menggabungkan semua term dari kolom "tweet" ke dalam list `term` tanpa adanya term yang berulang


In [9]:
term_frequency = []
# Membuat list kosong untuk menyimpan frekuensi term dalam setiap dokumen

countTF(df["tweet"], df["sentiment"], term, term_frequency)
# Memanggil fungsi `countTF` dengan parameter sebagai berikut:
# - `textColumn`: kolom "tweet" dari dataframe `df` yang berisi teks dalam setiap dokumen
# - `typeColumn`: kolom "sentiment" dari dataframe `df` yang berisi label sentimen untuk setiap dokumen
# - `termList`: list yang berisi semua term yang telah dihasilkan sebelumnya
# - `TFList`: list yang akan diisi dengan frekuensi term dalam setiap dokumen dan label sentimennya

print(len(term_frequency))
# Menampilkan panjang dari list `term_frequency`, yang mengindikasikan jumlah dokumen yang diproses


5938


In [10]:
tf_forDF = []
# Membuat list kosong untuk menyimpan frekuensi term dalam setiap dokumen dengan nilai yang telah diubah

changeValue(term_frequency, tf_forDF)
# Memanggil fungsi `changeValue` dengan parameter sebagai berikut:
# - `TFList`: list `term_frequency` yang berisi frekuensi term dalam setiap dokumen dan label sentimennya
# - `prepareDFList`: list yang akan diisi dengan frekuensi term dalam setiap dokumen dengan nilai yang telah diubah

print(len(tf_forDF))
# Menampilkan panjang dari list `tf_forDF`, yang mengindikasikan jumlah dokumen yang diproses setelah nilai yang lebih besar dari 1 diubah menjadi 1


5938


In [11]:
document_frequency = []
# Membuat list kosong untuk menyimpan frekuensi kemunculan setiap term dalam dokumen

countDF(tf_forDF, document_frequency)
# Memanggil fungsi `countDF` dengan parameter sebagai berikut:
# - `TFList`: list `tf_forDF` yang berisi frekuensi term dalam setiap dokumen dengan nilai yang telah diubah
# - `DFList`: list yang akan diisi dengan frekuensi kemunculan setiap term dalam dokumen

print(len(document_frequency))
# Menampilkan panjang dari list `document_frequency`, yang mengindikasikan jumlah term yang muncul dalam dokumen


14772


### **_Find value for RF & TF-RF_**

<ol>
  <li>RF</li>
  <li>TF-RF</li>

</ol>


In [12]:
# method to count TF in each document category
def count_tf_category(dataTableCategory, termList, textColumn):
    tf_spam_list = []
    for i in range(len(dataTableCategory)):
        doclist = []
        for j in termList:
            count = 0
            for k in range(len(textColumn[i])):
                if j == textColumn[i][k]:
                    count += 1
            doclist.append(count)
        doclist.append(dataTableCategory["sentiment"][i])
        tf_spam_list.append(doclist)
    return tf_spam_list


In [13]:
# splitting tweets that only contain positive sentiment
df_positive = df[df["sentiment"] != 0].reset_index(drop=True)
df_positive.head()


,tweet,sentiment
0,"[hadir, boss, mega, win, jackpot, jutaan, rupi...",1
1,"[hayolohhhhhhhhh, seneng, kannnn]",1
2,"[thank, you, temanteman, udah, mampir, ngobrol...",1
3,"[ruwet, ruwet, ruwet, inilah, negeri, ruwetnes...",1
4,"[film, bagus, nih, karya, habaib, tonton, dah]",1


In [15]:
# splitting tweets that only contain negative sentiment
df_negative = df[df["sentiment"] != 1].reset_index(drop=True)
df_negative.head()


,tweet,sentiment
0,"[stimulasi, optimal, dilakukan, online, memenu...",0
1,"[ye, ampun, mbh, seorang, wapres, lohampgelar,...",0
2,"[mulaik, kumat]",0
3,"[welcome, ppkm, bau, bau, naik, level, nih, gi...",0
4,"[sesungguhnya, allah, menciptakan, hambanya, m...",0


In [16]:
# count tf value in tweets with positive sentiment
tf_positive_rf = count_tf_category(df_positive, term, df_positive["tweet"])
print(len(tf_positive_rf))


1958


In [17]:
# count tf value in tweets with negative sentiment
tf_negative_rf = count_tf_category(df_negative, term, df_negative["tweet"])
print(len(tf_negative_rf))


3980


In [18]:
tf_positive_df = []
changeValue(tf_positive_rf, tf_positive_df)
print(len(tf_positive_df))


1958


In [19]:
tf_negative_df = []
changeValue(tf_negative_rf, tf_negative_df)
print(len(tf_negative_df))


3980


In [20]:
nij = []
countDF(tf_positive_df, nij)
print(len(nij))


14772


In [21]:
nij_aksen = []
countDF(tf_negative_df, nij_aksen)
print(len(nij_aksen))


14772


In [22]:
# determining the maximum value between 1 and the value of nij accent
max_numerator = []
for i in range(len(nij_aksen)):
    temp = 0
    temp = max(1, nij_aksen[i])
    max_numerator.append(temp)
print(max_numerator)


[5, 7, 18, 30, 14, 27, 18, 67, 9, 20, 66, 37, 18, 24, 50, 18, 5, 4, 3, 1, 1, 1, 5, 4, 1, 2, 2, 5, 1, 6, 1, 8, 1, 1, 7, 1, 189, 1, 1, 14, 7, 6, 36, 6, 10, 1, 1, 9, 31, 1, 2, 59, 72, 3, 2, 18, 1, 1, 1, 4, 142, 1, 196, 5, 12, 1, 15, 2, 1, 1, 1, 12, 144, 1, 22, 37, 2, 2, 1, 1, 9, 1, 9, 3, 52, 1, 1, 2, 5, 1067, 4, 43, 152, 117, 103, 4, 6, 37, 3, 3, 3, 7, 8, 1, 4, 57, 1, 1, 9, 30, 24, 21, 3, 96, 285, 120, 1, 51, 2, 37, 3, 1, 1, 37, 1, 1, 11, 1, 1, 1, 271, 24, 391, 1, 15, 6, 8, 2, 1, 2, 51, 37, 165, 9, 56, 1, 194, 32, 9, 434, 3, 2, 1, 1, 1, 2, 10, 2, 15, 40, 2, 23, 209, 1, 179, 19, 5, 46, 11, 18, 3, 62, 1, 1, 17, 62, 10, 2, 46, 4, 89, 9, 68, 75, 3, 25, 8, 1, 4, 24, 5, 1, 57, 21, 2, 1, 1, 1, 1, 2, 3, 5, 106, 7, 1, 25, 1, 1, 83, 1, 2, 1, 2, 1, 7, 42, 3, 89, 7, 2, 1, 1, 33, 3, 14, 8, 1, 16, 102, 44, 15, 1, 15, 1, 2, 9, 34, 39, 33, 14, 3, 9, 1, 23, 1, 5, 1, 7, 32, 10, 1, 4, 3, 1, 17, 1, 1, 1, 1, 14, 46, 4, 8, 10, 30, 2, 8, 1, 47, 4, 3, 66, 1, 207, 17, 7, 10, 5, 1, 144, 19, 2, 24, 10, 5, 128, 54, 

In [23]:
import math

# find rf value
rf_list = []
for i in range(len(nij)):
    temp = 0
    temp = math.log10(2 + (nij[i] / max_numerator[i]))
    temp = round(temp, 5)
    rf_list.append(temp)
print(rf_list)



[0.30103, 0.30103, 0.5006, 0.54818, 0.37239, 0.42597, 0.38818, 0.70026, 0.32451, 0.45484, 0.35364, 0.40028, 0.57724, 0.66901, 0.61278, 0.46073, 0.34242, 0.74036, 0.60206, 0.90309, 1.0, 0.90309, 0.5563, 0.67669, 1.0, 0.95424, 0.69897, 0.94448, 0.90309, 0.74036, 0.47712, 0.55931, 0.47712, 0.60206, 0.51663, 1.14613, 0.36302, 1.07918, 0.47712, 0.64626, 0.87091, 0.91204, 0.48509, 0.33579, 0.54407, 0.90309, 0.47712, 0.34679, 0.38945, 0.47712, 0.39794, 0.34309, 0.37312, 0.36798, 0.47712, 0.3782, 0.47712, 0.47712, 0.47712, 0.39794, 0.33059, 0.47712, 0.35021, 0.38021, 0.30103, 0.30103, 0.40369, 0.30103, 0.30103, 0.47712, 0.47712, 0.35218, 0.40984, 0.47712, 0.33882, 0.41857, 0.30103, 0.30103, 0.30103, 0.30103, 0.30103, 0.30103, 0.34679, 0.47712, 0.46297, 0.30103, 0.30103, 0.30103, 0.34242, 0.37358, 0.30103, 0.37085, 0.38518, 0.51388, 0.34127, 0.30103, 0.36798, 0.47712, 0.36798, 0.42597, 0.30103, 0.33099, 0.87506, 0.30103, 0.43933, 0.34793, 0.30103, 0.30103, 0.4437, 0.40369, 0.47105, 0.45593, 0.5

In [24]:
# Combining the values of TF for spam and non-spam.
tf_forRF = []

for i in range(len(tf_positive_rf)):
    tf_forRF.append(tf_positive_rf[i])

for i in range(len(tf_negative_rf)):
    tf_forRF.append(tf_negative_rf[i])

print(len(tf_forRF))

5938


In [25]:
tf_rf = []
for i in range(len(tf_forRF)):
    doclist = []
    for j in range(len(tf_forRF[0])):
        if j == len(tf_forRF[0]) - 1:
            temp = tf_forRF[i][j]
            doclist.append(temp)
        else:
            temp = rf_list[j] * tf_forRF[i][j]
            doclist.append(temp)
    tf_rf.append(doclist)
print(len(tf_rf))

5938


- **Save Dataframe To CSV**

In [26]:
columnName = term
columnName.append('sentiment')

In [27]:
data = pd.DataFrame(tf_rf, columns=columnName)
data.to_csv("../dataset/INA_TweetsPPKM_TFRF_DS1_NoStem.csv", index=False, sep="\t")


## **TF-RF Dataset 2 (Augmentasi)**

**1. Stemmed Dataset**

In [ ]:
load_path = "../dataset/INA_TweetsPPKM_Preprocessed_DS2.csv"
# Menentukan path file dataset yang akan di-load

df = pd.read_csv(load_path, sep="\t")
# Membaca dataset dari file CSV menggunakan pandas

df["tweet"] = df["tweet"].apply(lambda word: eval(word))
# Menggunakan fungsi `apply` pada kolom "tweet" untuk menerapkan fungsi lambda pada setiap kata
# Fungsi lambda digunakan untuk mengkonversi string kata-kata menjadi list kata-kata dengan menggunakan fungsi `eval`

df.sample(5)
# Menampilkan 10 baris pertama dari dataframe yang sudah diubah


In [ ]:
df.drop(['date','user'],axis=1,inplace=True,errors='ignore')

In [ ]:
term = []
# Membuat list kosong untuk menyimpan semua term

termMerging(termList=term, textColumn=df["tweet"])
# Memanggil fungsi `termMerging` dengan parameter `termList` yang merupakan list kosong yang akan diisi dengan semua term
# Parameter `textColumn` adalah kolom "tweet" dari dataframe `df`
# Fungsi `termMerging` akan menggabungkan semua term dari kolom "tweet" ke dalam list `term` tanpa adanya term yang berulang


In [ ]:
term_frequency = []
# Membuat list kosong untuk menyimpan frekuensi term dalam setiap dokumen

countTF(df["tweet"], df["sentiment"], term, term_frequency)
# Memanggil fungsi `countTF` dengan parameter sebagai berikut:
# - `textColumn`: kolom "tweet" dari dataframe `df` yang berisi teks dalam setiap dokumen
# - `typeColumn`: kolom "sentiment" dari dataframe `df` yang berisi label sentimen untuk setiap dokumen
# - `termList`: list yang berisi semua term yang telah dihasilkan sebelumnya
# - `TFList`: list yang akan diisi dengan frekuensi term dalam setiap dokumen dan label sentimennya

print(len(term_frequency))
# Menampilkan panjang dari list `term_frequency`, yang mengindikasikan jumlah dokumen yang diproses


In [ ]:
tf_forDF = []
# Membuat list kosong untuk menyimpan frekuensi term dalam setiap dokumen dengan nilai yang telah diubah

changeValue(term_frequency, tf_forDF)
# Memanggil fungsi `changeValue` dengan parameter sebagai berikut:
# - `TFList`: list `term_frequency` yang berisi frekuensi term dalam setiap dokumen dan label sentimennya
# - `prepareDFList`: list yang akan diisi dengan frekuensi term dalam setiap dokumen dengan nilai yang telah diubah

print(len(tf_forDF))
# Menampilkan panjang dari list `tf_forDF`, yang mengindikasikan jumlah dokumen yang diproses setelah nilai yang lebih besar dari 1 diubah menjadi 1


In [ ]:
document_frequency = []
# Membuat list kosong untuk menyimpan frekuensi kemunculan setiap term dalam dokumen

countDF(tf_forDF, document_frequency)
# Memanggil fungsi `countDF` dengan parameter sebagai berikut:
# - `TFList`: list `tf_forDF` yang berisi frekuensi term dalam setiap dokumen dengan nilai yang telah diubah
# - `DFList`: list yang akan diisi dengan frekuensi kemunculan setiap term dalam dokumen

print(len(document_frequency))
# Menampilkan panjang dari list `document_frequency`, yang mengindikasikan jumlah term yang muncul dalam dokumen


### **_Find value for RF & TF-RF_**

<ol>
  <li>RF</li>
  <li>TF-RF</li>

</ol>


In [ ]:
# method to count TF in each document category
def count_tf_category(dataTableCategory, termList, textColumn):
    tf_spam_list = []
    for i in range(len(dataTableCategory)):
        doclist = []
        for j in termList:
            count = 0
            for k in range(len(textColumn[i])):
                if j == textColumn[i][k]:
                    count += 1
            doclist.append(count)
        doclist.append(dataTableCategory["sentiment"][i])
        tf_spam_list.append(doclist)
    return tf_spam_list


In [ ]:
# splitting tweets that only contain positive sentiment
df_positive = df[df["sentiment"] != 0].reset_index(drop=True)
df_positive.head()


In [ ]:
# splitting tweets that only contain negative sentiment
df_negative = df[df["sentiment"] != 1].reset_index(drop=True)
df_negative.head()


In [ ]:
# count tf value in tweets with positive sentiment
tf_positive_rf = count_tf_category(df_positive, term, df_positive["tweet"])
print(len(tf_positive_rf))


In [ ]:
# count tf value in tweets with negative sentiment
tf_negative_rf = count_tf_category(df_negative, term, df_negative["tweet"])
print(len(tf_negative_rf))


In [ ]:
tf_positive_df = []
changeValue(tf_positive_rf, tf_positive_df)
print(len(tf_positive_df))


In [ ]:
tf_negative_df = []
changeValue(tf_negative_rf, tf_negative_df)
print(len(tf_negative_df))


In [ ]:
nij = []
countDF(tf_positive_df, nij)
print(len(nij))


In [ ]:
nij_aksen = []
countDF(tf_negative_df, nij_aksen)
print(len(nij_aksen))


In [ ]:
# determining the maximum value between 1 and the value of nij accent
max_numerator = []
for i in range(len(nij_aksen)):
    temp = 0
    temp = max(1, nij_aksen[i])
    max_numerator.append(temp)
print(max_numerator)


In [ ]:
import math

# find rf value
rf_list = []
for i in range(len(nij)):
    temp = 0
    temp = math.log10(2 + (nij[i] / max_numerator[i]))
    temp = round(temp, 5)
    rf_list.append(temp)
print(rf_list)


In [ ]:
# Combining the values of TF for spam and non-spam.
tf_forRF = []

for i in range(len(tf_positive_rf)):
    tf_forRF.append(tf_positive_rf[i])

for i in range(len(tf_negative_rf)):
    tf_forRF.append(tf_negative_rf[i])

print(len(tf_forRF))


In [ ]:
tf_rf = []
for i in range(len(tf_forRF)):
    doclist = []
    for j in range(len(tf_forRF[0])):
        if j == len(tf_forRF[0]) - 1:
            temp = tf_forRF[i][j]
            doclist.append(temp)
        else:
            temp = rf_list[j] * tf_forRF[i][j]
            doclist.append(temp)
    tf_rf.append(doclist)
print(len(tf_rf))


- **Save Dataframe To CSV**


In [ ]:
columnName = term
columnName.append('sentiment')

In [ ]:
data = pd.DataFrame(tf_rf, columns=columnName)
data.to_csv("../dataset/INA_TweetsPPKM_TFRF_DS3.csv", index=False, sep="\t")


**2. Non Stemmed Dataset**

In [30]:
load_path = "../dataset/INA_TweetsPPKM_Preprocessed_DS2_NoStem.csv"
# Menentukan path file dataset yang akan di-load

df = pd.read_csv(load_path, sep="\t")
# Membaca dataset dari file CSV menggunakan pandas

df["tweet"] = df["tweet"].apply(lambda word: eval(word))
# Menggunakan fungsi `apply` pada kolom "tweet" untuk menerapkan fungsi lambda pada setiap kata
# Fungsi lambda digunakan untuk mengkonversi string kata-kata menjadi list kata-kata dengan menggunakan fungsi `eval`

df.sample(5)
# Menampilkan 10 baris pertama dari dataframe yang sudah diubah


,tweet,sentiment
5392,"[bless, the, day, nyaman, banaar, ulun, gin, r...",0
3053,"[ppkm, udah, kaya, game, petualangan, aja, nih...",1
6929,"[ppkm, pak, presiden, kapan, mundur, pemilu, p...",0
4876,"[jangan, ppkm, doang, dong, pak, perpanjang, k...",0
3362,"[tahu, kripiko, tempe, cimol, kering, khas, ci...",1


In [31]:
df.drop(['date','user'],axis=1,inplace=True,errors='ignore')

In [32]:
term_frequency = []
# Membuat list kosong untuk menyimpan frekuensi term dalam setiap dokumen

countTF(df["tweet"], df["sentiment"], term, term_frequency)
# Memanggil fungsi `countTF` dengan parameter sebagai berikut:
# - `textColumn`: kolom "tweet" dari dataframe `df` yang berisi teks dalam setiap dokumen
# - `typeColumn`: kolom "sentiment" dari dataframe `df` yang berisi label sentimen untuk setiap dokumen
# - `termList`: list yang berisi semua term yang telah dihasilkan sebelumnya
# - `TFList`: list yang akan diisi dengan frekuensi term dalam setiap dokumen dan label sentimennya

print(len(term_frequency))
# Menampilkan panjang dari list `term_frequency`, yang mengindikasikan jumlah dokumen yang diproses


7896


In [33]:
tf_forDF = []
# Membuat list kosong untuk menyimpan frekuensi term dalam setiap dokumen dengan nilai yang telah diubah

changeValue(term_frequency, tf_forDF)
# Memanggil fungsi `changeValue` dengan parameter sebagai berikut:
# - `TFList`: list `term_frequency` yang berisi frekuensi term dalam setiap dokumen dan label sentimennya
# - `prepareDFList`: list yang akan diisi dengan frekuensi term dalam setiap dokumen dengan nilai yang telah diubah

print(len(tf_forDF))
# Menampilkan panjang dari list `tf_forDF`, yang mengindikasikan jumlah dokumen yang diproses setelah nilai yang lebih besar dari 1 diubah menjadi 1


7896


In [34]:
document_frequency = []
# Membuat list kosong untuk menyimpan frekuensi kemunculan setiap term dalam dokumen

countDF(tf_forDF, document_frequency)
# Memanggil fungsi `countDF` dengan parameter sebagai berikut:
# - `TFList`: list `tf_forDF` yang berisi frekuensi term dalam setiap dokumen dengan nilai yang telah diubah
# - `DFList`: list yang akan diisi dengan frekuensi kemunculan setiap term dalam dokumen

print(len(document_frequency))
# Menampilkan panjang dari list `document_frequency`, yang mengindikasikan jumlah term yang muncul dalam dokumen


14773


### **_Find value for RF & TF-RF_**

<ol>
  <li>RF</li>
  <li>TF-RF</li>

</ol>


In [35]:
# method to count TF in each document category
def count_tf_category(dataTableCategory, termList, textColumn):
    tf_spam_list = []
    for i in range(len(dataTableCategory)):
        doclist = []
        for j in termList:
            count = 0
            for k in range(len(textColumn[i])):
                if j == textColumn[i][k]:
                    count += 1
            doclist.append(count)
        doclist.append(dataTableCategory["sentiment"][i])
        tf_spam_list.append(doclist)
    return tf_spam_list


In [36]:
# splitting tweets that only contain positive sentiment
df_positive = df[df["sentiment"] != 0].reset_index(drop=True)
df_positive.head()


,tweet,sentiment
0,"[hadir, boss, mega, win, jackpot, jutaan, rupi...",1
1,"[hayolohhhhhhhhh, seneng, kannnn]",1
2,"[thank, you, temanteman, udah, mampir, ngobrol...",1
3,"[ruwet, ruwet, ruwet, inilah, negeri, ruwetnes...",1
4,"[film, bagus, nih, karya, habaib, tonton, dah]",1


In [37]:
# splitting tweets that only contain negative sentiment
df_negative = df[df["sentiment"] != 1].reset_index(drop=True)
df_negative.head()


,tweet,sentiment
0,"[stimulasi, optimal, dilakukan, online, memenu...",0
1,"[ye, ampun, mbh, seorang, wapres, lohampgelar,...",0
2,"[mulaik, kumat]",0
3,"[welcome, ppkm, bau, bau, naik, level, nih, gi...",0
4,"[sesungguhnya, allah, menciptakan, hambanya, m...",0


In [38]:
# count tf value in tweets with positive sentiment
tf_positive_rf = count_tf_category(df_positive, term, df_positive["tweet"])
print(len(tf_positive_rf))


3916


In [39]:
# count tf value in tweets with negative sentiment
tf_negative_rf = count_tf_category(df_negative, term, df_negative["tweet"])
print(len(tf_negative_rf))


3980


In [40]:
tf_positive_df = []
changeValue(tf_positive_rf, tf_positive_df)
print(len(tf_positive_df))


3916


In [41]:
tf_negative_df = []
changeValue(tf_negative_rf, tf_negative_df)
print(len(tf_negative_df))


3980


In [42]:
nij = []
countDF(tf_positive_df, nij)
print(len(nij))


14773


In [43]:
nij_aksen = []
countDF(tf_negative_df, nij_aksen)
print(len(nij_aksen))


14773


In [44]:
# determining the maximum value between 1 and the value of nij accent
max_numerator = []
for i in range(len(nij_aksen)):
    temp = 0
    temp = max(1, nij_aksen[i])
    max_numerator.append(temp)
print(max_numerator)


[5, 7, 18, 30, 14, 27, 18, 67, 9, 20, 66, 37, 18, 24, 50, 18, 5, 4, 3, 1, 1, 1, 5, 4, 1, 2, 2, 5, 1, 6, 1, 8, 1, 1, 7, 1, 189, 1, 1, 14, 7, 6, 36, 6, 10, 1, 1, 9, 31, 1, 2, 59, 72, 3, 2, 18, 1, 1, 1, 4, 142, 1, 196, 5, 12, 1, 15, 2, 1, 1, 1, 12, 144, 1, 22, 37, 2, 2, 1, 1, 9, 1, 9, 3, 52, 1, 1, 2, 5, 1067, 4, 43, 152, 117, 103, 4, 6, 37, 3, 3, 3, 7, 8, 1, 4, 57, 1, 1, 9, 30, 24, 21, 3, 96, 285, 120, 1, 51, 2, 37, 3, 1, 1, 37, 1, 1, 11, 1, 1, 1, 271, 24, 391, 1, 15, 6, 8, 2, 1, 2, 51, 37, 165, 9, 56, 1, 194, 32, 9, 434, 3, 2, 1, 1, 1, 2, 10, 2, 15, 40, 2, 23, 209, 1, 179, 19, 5, 46, 11, 18, 3, 62, 1, 1, 17, 62, 10, 2, 46, 4, 89, 9, 68, 75, 3, 25, 8, 1, 4, 24, 5, 1, 57, 21, 2, 1, 1, 1, 1, 2, 3, 5, 106, 7, 1, 25, 1, 1, 83, 1, 2, 1, 2, 1, 7, 42, 3, 89, 7, 2, 1, 1, 33, 3, 14, 8, 1, 16, 102, 44, 15, 1, 15, 1, 2, 9, 34, 39, 33, 14, 3, 9, 1, 23, 1, 5, 1, 7, 32, 10, 1, 4, 3, 1, 17, 1, 1, 1, 1, 14, 46, 4, 8, 10, 30, 2, 8, 1, 47, 4, 3, 66, 1, 207, 17, 7, 10, 5, 1, 144, 19, 2, 24, 10, 5, 128, 54, 

In [45]:
import math

# find rf value
rf_list = []
for i in range(len(nij)):
    temp = 0
    temp = math.log10(2 + (nij[i] / max_numerator[i]))
    temp = round(temp, 5)
    rf_list.append(temp)
print(rf_list)



[0.30103, 0.30103, 0.65854, 0.85126, 0.44494, 0.52768, 0.5708, 0.92597, 0.34679, 0.60746, 0.40317, 0.48102, 0.77815, 0.8653, 0.79796, 0.64235, 0.38021, 0.97772, 0.77815, 1.14613, 1.27875, 1.14613, 0.716, 0.8893, 1.27875, 1.21748, 0.90309, 1.23553, 1.14613, 1.06695, 0.60206, 0.72016, 0.60206, 0.8451, 0.7231, 1.14613, 0.41727, 1.34242, 0.47712, 0.84064, 1.17609, 1.18564, 0.70377, 0.4523, 0.69897, 1.14613, 0.60206, 0.47712, 0.50864, 0.60206, 0.47712, 0.40233, 0.44802, 0.47712, 0.60206, 0.57724, 0.60206, 0.60206, 0.60206, 0.51188, 0.3596, 0.60206, 0.40147, 0.47712, 0.30103, 0.30103, 0.5563, 0.30103, 0.30103, 0.69897, 0.60206, 0.39794, 0.5165, 0.60206, 0.37358, 0.54574, 0.30103, 0.30103, 0.30103, 0.30103, 0.32451, 0.47712, 0.47712, 0.60206, 0.58935, 0.30103, 0.30103, 0.30103, 0.38021, 0.62302, 0.30103, 0.43841, 0.46943, 0.65527, 0.37633, 0.35218, 0.42597, 0.61932, 0.42597, 0.42597, 0.30103, 0.35902, 1.18682, 0.30103, 0.62839, 0.41731, 0.30103, 0.30103, 0.60206, 0.63682, 0.63263, 0.60206, 0.

In [46]:
# Combining the values of TF for spam and non-spam.
tf_forRF = []

for i in range(len(tf_positive_rf)):
    tf_forRF.append(tf_positive_rf[i])

for i in range(len(tf_negative_rf)):
    tf_forRF.append(tf_negative_rf[i])

print(len(tf_forRF))

7896


In [47]:
tf_rf = []
for i in range(len(tf_forRF)):
    doclist = []
    for j in range(len(tf_forRF[0])):
        if j == len(tf_forRF[0]) - 1:
            temp = tf_forRF[i][j]
            doclist.append(temp)
        else:
            temp = rf_list[j] * tf_forRF[i][j]
            doclist.append(temp)
    tf_rf.append(doclist)
print(len(tf_rf))

7896


- **Save Dataframe To CSV**

In [48]:
columnName = term
columnName.append('sentiment')

In [49]:
data = pd.DataFrame(tf_rf, columns=columnName)
data.to_csv("../dataset/INA_TweetsPPKM_TFRF_DS2_NoStem.csv", index=False, sep="\t")
